### Equitable Model

In [5]:
# baseball_assignment.py

import pandas as pd
import pulp

# Mapping of position abbreviations to full names
position_mapping = {
    'P': 'Pitcher',
    'C': 'Catcher',
    '1B': 'First Base',
    '2B': 'Second Base',
    '3B': 'Third Base',
    'SS': 'Shortstop',
    'LF': 'Left Field',
    'CF': 'Center Field',
    'RF': 'Right Field',
    # Add any other position abbreviations as needed
}

# Read player data from Excel file
# The Excel file 'player_data.xlsx' should have columns:
# 'First', 'Last', 'Full Name', 'Position', 'Rating'

player_data = pd.read_excel('../Source/Defensive Rankings.xlsx', sheet_name='Defensive Ratings Extended')

# Map position abbreviations to full names
player_data['Position'] = player_data['Position'].map(position_mapping)

# Ensure 'Rating' is numeric
player_data['Rating'] = pd.to_numeric(player_data['Rating'], errors='coerce')

# Build the players dictionary
# Each player has 'positions' and 'innings_played'

players = {}
for player in player_data['Full Name'].unique():
    positions = player_data[player_data['Full Name'] == player][['Position', 'Rating']].set_index('Position')['Rating'].to_dict()
    players[player] = {
        'positions': positions,
        'innings_played': 0,
    }

total_innings = 9  # Total innings in the game

# Ask the user for pitcher information
num_pitchers = int(input("Enter the number of pitchers for the game: "))

# Get list of players who can play as Pitcher
available_pitchers = [player for player in players if 'Pitcher' in players[player]['positions']]

if not available_pitchers:
    print("Error: No available pitchers found. Please ensure that there are players eligible for 'Pitcher' position.")
    exit(1)

print("\nAvailable pitchers:")
for idx, pitcher in enumerate(available_pitchers, 1):
    print(f"{idx}. {pitcher}")

pitcher_schedule = {}
innings_left = total_innings

for i in range(num_pitchers):
    pitcher_input = input(f"\nEnter the number or full name of pitcher {i+1}: ").strip()
    # Check if input is a number
    if pitcher_input.isdigit():
        pitcher_index = int(pitcher_input) - 1
        if 0 <= pitcher_index < len(available_pitchers):
            pitcher_name = available_pitchers[pitcher_index]
        else:
            print("Invalid selection.")
            exit(1)
    else:
        pitcher_name = pitcher_input
        if pitcher_name not in available_pitchers:
            print(f"Error: {pitcher_name} is not an available pitcher.")
            exit(1)

    if i == num_pitchers - 1:
        innings = innings_left  # Remaining innings
    else:
        innings = int(input(f"Enter the number of innings {pitcher_name} will pitch (max {innings_left}): "))
        if innings > innings_left or innings <= 0:
            print(f"Error: Invalid number of innings. {innings_left} innings remaining.")
            exit(1)
        innings_left -= innings
    pitcher_schedule[pitcher_name] = innings

if innings_left > 0:
    print(f"Error: Not all innings assigned to pitchers. {innings_left} innings remaining.")
    exit(1)

# Build pitcher assignments per inning
inning_pitcher = {}
current_inning = 1
for pitcher_name, innings in pitcher_schedule.items():
    for _ in range(innings):
        inning_pitcher[current_inning] = pitcher_name
        current_inning += 1

# Identify when the pitcher changes
pitcher_change = {}
for inning in range(1, total_innings):
    pitcher_this_inning = inning_pitcher[inning]
    pitcher_next_inning = inning_pitcher[inning + 1]
    pitcher_change[inning] = 1 if pitcher_this_inning != pitcher_next_inning else 0

# Initialize the optimization problem
prob = pulp.LpProblem("Baseball_Position_Assignment", pulp.LpMaximize)

# List of all positions
positions = [
    'Pitcher',
    'Catcher',
    'First Base',
    'Second Base',
    'Third Base',
    'Shortstop',
    'Left Field',
    'Center Field',
    'Right Field',
]

# Decision variables
x = {}  # x[(inning, player, position)] = 1 if player plays position in inning

# Create decision variables with pipe '|' as delimiter
for inning in range(1, total_innings + 1):
    for player in players:
        for position in players[player]['positions']:
            var_name = f"x|{inning}|{player}|{position}"
            x[(inning, player, position)] = pulp.LpVariable(var_name, 0, 1, pulp.LpBinary)

# Fix the assigned pitchers
for inning in range(1, total_innings + 1):
    pitcher_name = inning_pitcher[inning]
    x[(inning, pitcher_name, 'Pitcher')].setInitialValue(1)
    x[(inning, pitcher_name, 'Pitcher')].fixValue()
    # The pitcher cannot play any other position in this inning
    for position in players[pitcher_name]['positions']:
        if position != 'Pitcher':
            x[(inning, pitcher_name, position)].setInitialValue(0)
            x[(inning, pitcher_name, position)].fixValue()
    # No other player can play 'Pitcher' in this inning
    for other_player in players:
        if other_player != pitcher_name and 'Pitcher' in players[other_player]['positions']:
            x[(inning, other_player, 'Pitcher')].setInitialValue(0)
            x[(inning, other_player, 'Pitcher')].fixValue()

# Identify positions that only one player can play (excluding 'Pitcher')
position_players = {}
for position in positions:
    players_for_position = [player for player in players if position in players[player]['positions']]
    position_players[position] = players_for_position

# Check for positions with no eligible players
positions_with_no_players = [position for position, player_list in position_players.items() if len(player_list) == 0]
if positions_with_no_players:
    print("Error: The following positions have no eligible players:")
    for position in positions_with_no_players:
        print(f"- {position}")
    print("Please ensure that there are eligible players for all positions.")
    exit(1)

positions_with_one_player = [position for position, player_list in position_players.items() if len(player_list) == 1 and position != 'Pitcher']

# Fix assignments for positions that only one player can play
for position in positions_with_one_player:
    player = position_players[position][0]
    for inning in range(1, total_innings + 1):
        x[(inning, player, position)].setInitialValue(1)
        x[(inning, player, position)].fixValue()
        # The player cannot play any other position in this inning
        for other_position in players[player]['positions']:
            if other_position != position and other_position != 'Pitcher':
                x[(inning, player, other_position)].setInitialValue(0)
                x[(inning, player, other_position)].fixValue()
        # No other player can play this position in this inning
        for other_player in players:
            if other_player != player and position in players[other_player]['positions']:
                x[(inning, other_player, position)].setInitialValue(0)
                x[(inning, other_player, position)].fixValue()

# Identify the starting catcher (highest-rated catcher)
catchers = position_players['Catcher']
if not catchers:
    print("Error: No available catchers found.")
    exit(1)

# Select the starting catcher based on the highest rating
starting_catcher = max(catchers, key=lambda player: players[player]['positions']['Catcher'])

# Fix the starting catcher's assignment for the first 4 innings
for inning in range(1, 5):  # Innings 1 to 4
    x[(inning, starting_catcher, 'Catcher')].setInitialValue(1)
    x[(inning, starting_catcher, 'Catcher')].fixValue()
    # The starting catcher cannot play other positions during these innings
    for position in players[starting_catcher]['positions']:
        if position != 'Catcher' and position != 'Pitcher':
            x[(inning, starting_catcher, position)].setInitialValue(0)
            x[(inning, starting_catcher, position)].fixValue()

# Objective function: Maximize total ratings minus penalties for soft constraint violations
objective_terms = []
penalty_terms = []
penalty_weights = {
    'Outfielders': 1000,
    'Infielders': 1000,
    'Defensive_Avg': 1000,
    'Defensive_Pitcher_Avg': 1000,
    '1B_2B': 500,
    '2B_SS': 500,
    'SS_3B': 500,
    'Pitcher_Rest': 100,  # Penalty weight for pitcher not resting before/after pitching
    'Catcher_Change': 1000,  # Penalty weight for catcher changes when pitcher doesn't change
}

for inning in range(1, total_innings + 1):
    for player in players:
        for position in players[player]['positions']:
            rating = players[player]['positions'][position]
            objective_terms.append(rating * x[(inning, player, position)])

# Constraints

# Each position must be filled once per inning (excluding 'Pitcher' since it's fixed)
for inning in range(1, total_innings + 1):
    for position in positions:
        if position == 'Pitcher':
            continue
        eligible_players = [player for player in players if position in players[player]['positions']]
        if not eligible_players:
            print(f"Error: No eligible players for position '{position}'. Cannot proceed.")
            exit(1)
        prob += pulp.lpSum([x[(inning, player, position)] for player in eligible_players]) == 1, f"Position_{position}_Inning_{inning}"

# Each player plays at most one position per inning
for inning in range(1, total_innings + 1):
    for player in players:
        prob += pulp.lpSum([x[(inning, player, position)] for position in players[player]['positions']]) <= 1, f"Player_{player}_Inning_{inning}"

# Fair playing time constraints
num_players = len(players)

# Adjust max innings based on number of players
if num_players >= 18:
    max_inn = total_innings - 3  # 6 innings
elif num_players >= 15:
    max_inn = total_innings - 2  # 7 innings
elif num_players >= 13:
    max_inn = total_innings - 1  # 8 innings
else:
    max_inn = total_innings      # 9 innings

min_inn = 4  # Every player must play at least 4 innings

for player in players:
    innings_as_pitcher = sum(1 for inning in range(1, total_innings + 1) if inning_pitcher.get(inning) == player)
    total_innings_played = (
        pulp.lpSum([x[(inning, player, position)] for inning in range(1, total_innings + 1)
                    for position in players[player]['positions'] if position != 'Pitcher']) + innings_as_pitcher
    )
    prob += total_innings_played >= min_inn, f"MinInnings_{player}"
    prob += total_innings_played <= max_inn, f"MaxInnings_{player}"

# Soft Constraints Implementation

# ... [Soft constraints from previous script remain unchanged]

# [Insert soft constraints code here, same as previous script]

# --- New Constraints for Pitcher's Rest Before and After Pitching ---

for pitcher_name in pitcher_schedule:
    # Get innings this pitcher is pitching
    pitching_innings = [inning for inning in range(1, total_innings + 1) if inning_pitcher[inning] == pitcher_name]
    for inning in pitching_innings:
        # Inning before
        if inning > 1:
            prev_inning = inning - 1
            # Sum of x variables for all positions except 'Pitcher' in prev_inning
            plays_before = pulp.lpSum([
                x[(prev_inning, pitcher_name, pos)] for pos in players[pitcher_name]['positions'] if pos != 'Pitcher'
            ])
            # Penalty if pitcher plays in the inning before pitching
            penalty_terms.append(-penalty_weights['Pitcher_Rest'] * plays_before)
        # Inning after
        if inning < total_innings:
            next_inning = inning + 1
            # Sum of x variables for all positions except 'Pitcher' in next_inning
            plays_after = pulp.lpSum([
                x[(next_inning, pitcher_name, pos)] for pos in players[pitcher_name]['positions'] if pos != 'Pitcher'
            ])
            # Penalty if pitcher plays in the inning after pitching
            penalty_terms.append(-penalty_weights['Pitcher_Rest'] * plays_after)

# --- Constraints for Catcher Swap with Pitcher Changes ---

# For innings where pitcher does not change, enforce that the catcher does not change
for inning in range(1, total_innings):
    if pitcher_change[inning] == 0:
        # Pitcher does not change between inning and inning +1
        for catcher in catchers:
            # The difference in catcher assignment between inning and inning +1 should be zero
            prob += x[(inning, catcher, 'Catcher')] - x[(inning + 1, catcher, 'Catcher')] == 0, f"Catcher_Consistency_{catcher}_Inning_{inning}"
    else:
        # Pitcher changes, catcher can change
        pass  # No constraint needed

# --- End of New Constraints ---

# Combine objective function terms
prob += pulp.lpSum(objective_terms) + pulp.lpSum(penalty_terms), "TotalObjective"

# Solve the problem
prob.solve()

print("Status:", pulp.LpStatus[prob.status])

if pulp.LpStatus[prob.status] != 'Optimal':
    print("No optimal solution found.")
    exit(1)

# Retrieve the results
assignments = []
for v in prob.variables():
    if v.varValue == 1 and v.name.startswith('x|'):
        var_name = v.name
        # var_name format: x|inning|player|position
        _, inning, player, position = var_name.split('|', 3)
        inning = int(inning)
        # Replace underscores with spaces (PuLP replaces spaces with underscores)
        player = player.replace('_', ' ')
        position = position.replace('_', ' ')
        assignments.append({
            'Inning': inning,
            'Position': position,
            'Player': player
        })
        players[player]['innings_played'] += 1

# Add pitcher assignments to the assignments list
for inning in range(1, total_innings + 1):
    pitcher_name = inning_pitcher[inning]
    assignments.append({
        'Inning': inning,
        'Position': 'Pitcher',
        'Player': pitcher_name
    })
    players[pitcher_name]['innings_played'] += 1

# Create a DataFrame from assignments
assignments_df = pd.DataFrame(assignments)

# Check for missing assignments
missing_assignments = []
for inning in range(1, total_innings + 1):
    for position in positions:
        assigned_players = assignments_df[(assignments_df['Inning'] == inning) & (assignments_df['Position'] == position)]
        if assigned_players.empty:
            missing_assignments.append({'Inning': inning, 'Position': position})

if missing_assignments:
    print("Warning: Missing assignments detected for the following positions:")
    for item in missing_assignments:
        print(f"Inning {item['Inning']}, Position {item['Position']}")

# Pivot the DataFrame to have positions as columns
lineup_df = assignments_df.pivot_table(index='Inning', columns='Position', values='Player', aggfunc='first')
lineup_df = lineup_df[positions]  # Ensure correct column order
lineup_df = lineup_df.sort_index()

# Display innings played per player
innings_played_df = pd.DataFrame({
    'Player': list(players.keys()),
    'Innings_Played': [players[player]['innings_played'] for player in players]
})



Available pitchers:
1. Alex Markey
2. Jack Barbash
3. Nick Alexander
4. Stephen Garrett
5. Garrett Beloff
6. Brad Ellis
7. Joe Vanderplas
8. Ryan Shea
9. Joe Bruchalski
10. Robert Robbins
Error: Not all innings assigned to pitchers. 2 innings remaining.
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/test/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xb/kvztmq1j2mz23r64f2dbx0mh0000gp/T/72b60f8c5e004c3fbfe937c1b6b493e9-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/xb/kvztmq1j2mz23r64f2dbx0mh0000gp/T/72b60f8c5e004c3fbfe937c1b6b493e9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 288 COLUMNS
At line 3595 RHS
At line 3879 BOUNDS
At line 4384 ENDATA
Problem MODEL has 283 rows, 504 columns and 1794 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6450 - 0.00

In [6]:
lineup_df

Position,Pitcher,Catcher,First Base,Second Base,Third Base,Shortstop,Left Field,Center Field,Right Field
Inning,,,,,,,,,
1,Alex Markey,Trevor Doucet,Nick Alexander,Ted Engelhardt,Vinnie Heher,Brad Ellis,Joe Bruchalski,Stephen Garrett,Chris Leiss
2,Alex Markey,Trevor Doucet,Nick Alexander,Ted Engelhardt,Jacob Landa,Seamus Quirk,Joe Bruchalski,Stephen Garrett,Joe Vanderplas
3,Alex Markey,Trevor Doucet,Jack Barbash,Garrett Beloff,Vinnie Heher,Seamus Quirk,Joe Bruchalski,Stephen Garrett,Chris Leiss
4,Alex Markey,Trevor Doucet,Nick Alexander,Garrett Beloff,Jacob Landa,Seamus Quirk,Ryan Shea,Stephen Garrett,Chris Leiss
5,Jack Barbash,Patrick Holleran,Stephen Garrett,Garrett Beloff,Vinnie Heher,Seamus Quirk,Ryan Shea,Joe Vanderplas,Robert Robbins
6,Jack Barbash,Patrick Holleran,Alex Markey,Garrett Beloff,Jacob Landa,Seamus Quirk,Joe Bruchalski,Stephen Garrett,Robert Robbins
7,Jack Barbash,Patrick Holleran,Nick Alexander,Ted Engelhardt,Vinnie Heher,Brad Ellis,Joe Bruchalski,Stephen Garrett,Chris Leiss
8,Ryan Shea,Patrick Holleran,Alex Markey,Brad Ellis,Jacob Landa,Seamus Quirk,Joe Bruchalski,Joe Vanderplas,Robert Robbins
9,Ryan Shea,Patrick Holleran,Alex Markey,Ted Engelhardt,Brad Ellis,Seamus Quirk,Joe Bruchalski,Joe Vanderplas,Robert Robbins
